## Load configurations

In [ ]:
from cfgs.content_classification_1_output import get_cfg

cfgs = get_cfg()
print('\n', cfgs, '\n')

In [ ]:
dataset_dict = cfgs.CLASSES[0]
dataset_dict['Genre_alias'] = dict((g, i) for i, g in dataset_dict['Genre'].items())

## Load data and initialize generator

In [ ]:
import pandas as pd
from data_processing.data_generator import DataGenerator
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_pickle("/home/robotics/content-classification/data_processing/balanced_data.pkl")

In [ ]:
data_generator = DataGenerator(df, dataset_dict, cfgs.HYPERPARAMS.TEST_SPLIT, cfgs.HYPERPARAMS.VAL_SPLIT, cfgs.IMAGE.RESOLUTION)
data_generator.df.head()

In [ ]:
train_idx, valid_idx, test_idx = data_generator.split_dataset() 

## Build and compile model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from model.architecture import MultiOutputModel

In [ ]:
structure = cfgs.MODEL.STRUCTURE[0]
content_model = MultiOutputModel(*cfgs.IMAGE.RESOLUTION, structure)
content_model.build_model("InceptionV3")
# keras.utils.plot_model(content_model.model, show_shapes=True)

In [ ]:
# opt = SGD()
opt = Adam(lr=cfgs.HYPERPARAMS.LR, decay=cfgs.HYPERPARAMS.LR/cfgs.HYPERPARAMS.EPOCHS)
content_model.compile_model(opt)
content_model.model.summary()

## Train Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.backend import clear_session
import tensorflow as tf

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

# GPU check - CUDA 11.1 with cuDNN 8.05 on 460.39 - GTX 1060 6GB
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

clear_session()

In [ ]:
tr_generator = data_generator.generate_images(train_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TR, True)
val_generator = data_generator.generate_images(valid_idx, cfgs.HYPERPARAMS.BATCH_SIZE_VAL, True)

In [ ]:
import datetime

%load_ext tensorboard
log_dir = log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)]
# keras.callbacks.ModelCheckpoint('./checkpoints', monitor='val_loss', verbose=0, 
#                                              save_best_only=False, save_weights_only=False, 
#                                              mode='auto', save_freq='epoch')

In [ ]:
%%time
history = content_model.model.fit(tr_generator, 
                    steps_per_epoch=len(train_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_TR,
                    epochs=cfgs.HYPERPARAMS.EPOCHS,
                    callbacks=callbacks,
                    validation_data=val_generator,
                    validation_steps=len(valid_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_VAL)

## TensorBoard

In [ ]:
%tensorboard --logdir logs

In [ ]:
content_model.model.save("Model_v5")

## Test set

In [ ]:
import numpy as np

In [ ]:
t_gen = data_generator.generate_images(test_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TEST, False)
genre_inf = content_model.model.predict(t_gen, steps=len(test_idx)//cfgs.HYPERPARAMS.BATCH_SIZE_TEST)

genres_pred = genre_inf.argmax(axis=-1)

In [ ]:
genres_true = data_generator.generate_truth(test_idx, cfgs.HYPERPARAMS.BATCH_SIZE_TEST)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
cr_genre = classification_report(genres_true, genres_pred, target_names=dataset_dict['Genre_alias'].keys())
print(cr_genre)

In [ ]:
from sklearn.metrics import r2_score
print('R2 score for ratings: ', r2_score(ratings_true, ratings_pred))